# <font color="blue">Submitted by: Kaspar Kadalipp </font>
# HW12. Suffix trees/arrays, Peer review, Feedback, TSP competition

### <font color='orange'> Less important code is placed here</font>
### <font color='orange'> Report is below </font>

In [1]:
import math
inf = float('inf')

def euclideanDistance(node1, node2):
    x1, y1 = node1
    x2, y2 = node2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# from https://github.com/Retsediv/ChristofidesAlgorithm
def tsp(data):
    G = build_graph(data)
    MSTree = minimum_spanning_tree(G)
    odd_vertexes = find_odd_vertexes(MSTree)
    minimum_weight_matching(MSTree, G, odd_vertexes)
    eulerian_tour = find_eulerian_tour(MSTree, G)

    current = eulerian_tour[0]
    path = [current]
    visited = [False] * len(eulerian_tour)
    visited[eulerian_tour[0]] = True
    length = 0

    for v in eulerian_tour:
        if not visited[v]:
            path.append(v)
            visited[v] = True

            length += G[current][v]
            current = v

    length += G[current][eulerian_tour[0]]
    return path, length


def get_length(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** (1.0 / 2.0)


def build_graph(data):
    graph = {}
    for this in range(len(data)):
        for another_point in range(len(data)):
            if this != another_point:
                if this not in graph:
                    graph[this] = {}
                graph[this][another_point] = get_length(data[this][0], data[this][1], data[another_point][0],data[another_point][1])

    return graph


class UnionFind:
    def __init__(self):
        self.weights = {}
        self.parents = {}

    def __getitem__(self, object):
        if object not in self.parents:
            self.parents[object] = object
            self.weights[object] = 1
            return object

        # find path of objects leading to the root
        path = [object]
        root = self.parents[object]
        while root != path[-1]:
            path.append(root)
            root = self.parents[root]

        # compress the path and return
        for ancestor in path:
            self.parents[ancestor] = root
        return root

    def __iter__(self):
        return iter(self.parents)

    def union(self, *objects):
        roots = [self[x] for x in objects]
        heaviest = max([(self.weights[r], r) for r in roots])[1]
        for r in roots:
            if r != heaviest:
                self.weights[heaviest] += self.weights[r]
                self.parents[r] = heaviest


def minimum_spanning_tree(G):
    tree = []
    subtrees = UnionFind()
    for W, u, v in sorted((G[u][v], u, v) for u in G for v in G[u]):
        if subtrees[u] != subtrees[v]:
            tree.append((u, v, W))
            subtrees.union(u, v)

    return tree


def find_odd_vertexes(MST):
    tmp_g = {}
    vertexes = []
    for edge in MST:
        if edge[0] not in tmp_g:
            tmp_g[edge[0]] = 0

        if edge[1] not in tmp_g:
            tmp_g[edge[1]] = 0

        tmp_g[edge[0]] += 1
        tmp_g[edge[1]] += 1

    for vertex in tmp_g:
        if tmp_g[vertex] % 2 == 1:
            vertexes.append(vertex)

    return vertexes


def minimum_weight_matching(MST, G, odd_vert):
    import random
    random.shuffle(odd_vert)

    while odd_vert:
        v = odd_vert.pop()
        length = float("inf")
        u = 1
        closest = 0
        for u in odd_vert:
            if v != u and G[v][u] < length:
                length = G[v][u]
                closest = u

        MST.append((v, closest, length))
        odd_vert.remove(closest)


def find_eulerian_tour(MatchedMSTree, G):
    # find neigbours
    neighbours = {}
    for edge in MatchedMSTree:
        if edge[0] not in neighbours:
            neighbours[edge[0]] = []

        if edge[1] not in neighbours:
            neighbours[edge[1]] = []

        neighbours[edge[0]].append(edge[1])
        neighbours[edge[1]].append(edge[0])

    # print("Neighbours: ", neighbours)

    # finds the hamiltonian circuit
    start_vertex = MatchedMSTree[0][0]
    EP = [neighbours[start_vertex][0]]

    while len(MatchedMSTree) > 0:
        for i, v in enumerate(EP):
            if len(neighbours[v]) > 0:
                break

        while len(neighbours[v]) > 0:
            w = neighbours[v][0]

            remove_edge_from_matchedMST(MatchedMSTree, v, w)

            del neighbours[v][(neighbours[v].index(w))]
            del neighbours[w][(neighbours[w].index(v))]

            i += 1
            EP.insert(i, w)
            v = w

    return EP


def remove_edge_from_matchedMST(MatchedMST, v1, v2):
    for i, item in enumerate(MatchedMST):
        if (item[0] == v2 and item[1] == v1) or (item[0] == v1 and item[1] == v2):
            del MatchedMST[i]
    return MatchedMST

def path_improvement(distance_matrix, vertices, prev_path, prev_distance):
    best_path = prev_path
    best_distance = prev_distance
    while True:
        prev_best = best_distance
        for i in range(len(prev_path)):
            for j in range(len(prev_path)):
                path = best_path[:]
                path[i], path[j] = path[j], path[i]
                distance = 0
                for k in range(1, len(path)):
                    distance += distance_matrix[path[k - 1]][path[k]][0]
                distance += euclideanDistance(vertices[path[0]], vertices[path[-1]])
                if distance < best_distance:
                    best_path = path
                    best_distance = distance
        if best_distance == prev_best:
            break
    return best_path, best_distance

# https://codereview.stackexchange.com/questions/208387/2-opt-algorithm-for-the-traveling-salesman-and-or-sro
def two_opt(cost_mat, route, vertices):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1:
                    continue  # changes nothing, skip then
                new_route = route[:]    # Creates a copy of route
                new_route[i:j] = route[j - 1:i - 1:-1]  # this is the 2-optSwap since j >= i we use -1
                c = cost(cost_mat, new_route, vertices)
                if c < cost(cost_mat, route, vertices):
                    route = new_route    # change current route to best
                    if c < 24800:
                        print("Improved to:", c)
                    improved = True
    return route, cost(cost_mat, route, vertices)

def cost(distance_matrix, path, vertices):
    distance = 0
    for k in range(1, len(path)):
        distance += distance_matrix[path[k - 1]][path[k]][0]
    distance += euclideanDistance(vertices[path[0]], vertices[path[-1]])
    return distance

def ex6(size):
    with open(f'{size}.txt', 'r') as f:
        vertices = [tuple(int(val) for val in line.strip().split(" ")) for line in f.readlines()]
        distance_matrix = [[0 for col in range(len(vertices))] for row in range(len(vertices))]
        for row in range(len(vertices)):
            for col in range(row, len(vertices)):
                ed = euclideanDistance(vertices[row], vertices[col])
                distance_matrix[row][col] = (ed, col)
                distance_matrix[col][row] = (ed, row)

        path, length = tsp(vertices)
        print(f'Christofides algorithm distance: {length:.2f}')
        path, length = two_opt(distance_matrix, path, vertices)
        print(f'Distance after 2-opt improvement: {length:.2f}') # 25704.24
        with open(f'{size}_optimised.txt', 'w+') as f:
            for index in path:
                f.write(f'{index}\n')

# EX1 Peer review the first article assigned to you.

#### reviewed student B47904 ✅

# EX2 Peer review the second article assigned to you.

#### reviewed student B53231 ✅

# EX3

##### You are given word: sinsinati.

##### What are all the suffixes for the given word? Build a Suffix Trie and a Tree for the same word. Demonstrate searching for pattern "sina" using these data structures. Make sure to describe the algorithms you used to perform these tasks. What is a Suffix Trie/Tree be used for? What is the task of full-text indexing? What are the differences between using Tree instead of Trie?

##### Take the same word and instead build a Suffix Array. Describe the algorithm. How could you use this suffix array to perform some operations? Where should we use Suffix Arrays instead of Suffix trees?

All suffixes:
- sinsinati
- insinati
- nsinati
- sinati
- inati
- nati
- ati
- ti
- i


Trie construction starts by iterating through characters of the text.
It first builds trie $T_1$ using 1st character, then $T_2$ using 2nd character, then $T_3$ using 3rd character, …, $T_m$ using m-th character.
Suffix trie $T_{i+1}$ is built on top of suffix tree $T_i$ by extending leaf nodes to be sure the suffix is in the trie.

Suffix tree is very similar to trie, here all nodes with one child are simply merged with their parents.

Suffix trie/tree are used to find occurrences of a pattern in a string. By preprocessing the text in $O(n)$ time a pattern can be found in $O(m)$ complexity, where $m$ is the length of the pattern and $n$ length of the text.
Also, they can be used for finding the longest repeated substring, the longest common substring, the longest palindrome.

Full-text indexing builds a data structure from the text which allows to efficiently find patterns. It splits the pattern searching into two phases, counting the number of pattern occurrences and locating their positions.



Suffix array is gotten from sorting all suffixes of a string alphabetically and remembering the starting positions of these sorted suffixes.


Suffix arrays take less space than suffix trees and any suffix tree algorithm can be replaced with an algorithm that uses a suffix array enhanced with additional information.

The suffix array of a string can be used as an index to quickly locate every occurrence of a substring pattern P within the string S. Finding every occurrence of the pattern is equivalent to finding every suffix that begins with the substring. Thanks to the lexicographical ordering, these suffixes will be grouped together in the suffix array and can be found efficiently with two binary searches. The first search locates the starting position of the interval, and the second one determines the end position:


<font color="gray" >used resource: https://en.wikipedia.org/wiki/Suffix_array </font>

![ex3](https://i.imgur.com/P2nbFHL.png)
![searching pattern](https://i.imgur.com/WM3zXUO.png)

# EX4

##### Substring matching. Given two strings S1 and S2, and a positive integer k, find the number of substrings of S1 of length at least k that occur in S2. Develop and analyze an algorithm to solve this problem in O(|S1| + |S2| + sort(Σ)) time, where Σ is the alphabet.

We can solve this problem in O(|S1| + |S2| + sort(Σ)) time using a suffix array. The suffix array is a data structure that stores all the suffixes of a given string in an array. We can then use this array to search for substrings of S1 of length at least k that occur in S2.

To begin, we create a suffix array for S1 and S2. We then traverse through the suffix array, and for each suffix of S1, we search for it in S2. If the suffix of S1 is found in S2, we check to see if its length is at least k. If so, we increment a counter. We repeat this process for all suffixes of S1 and S2.

The resulting algorithm runs in O(|S1| + |S2| + sort(Σ)) time, since we need to construct the suffix array, which takes O(|S1| + sort(Σ)) time, and then traverse through the suffix array, which takes O(|S2|) time.

# EX5

##### Perform Burrows-Wheeler Transform on the following string: CGGTCGCT$. Make sure to draw the matrix. Demonstrate how you can reconstruct the original string from BWT. Where and why is BWT used? This <a href="https://www.youtube.com/watch?v=P3ORBMon8aw">video</a> might help you get started:




 Burrows–Wheeler transform is useful for lossless compression since it restructures data in such a way that the transformed text is more compressible. The transformation is reversible, without needing to store any additional data except the position of the first original character. The algorithm can be implemented efficiently using a suffix array thus reaching linear time complexity.

It has applications in image compression and compression of genomic databases. It's also used in genetics sequence alignment and for sequence prediction in machine learning models.


<font color="gray" >used resource: https://en.wikipedia.org/wiki/Burrows%E2%80%93Wheeler_transform</font>

![ex6.1](https://i.imgur.com/FZFnvjj.png)
![ex6.2](https://i.imgur.com/CUpPzRF.png)
![ex6.3](https://i.imgur.com/9kDWjZb.png)

# EX6

##### As was promised, the last task is from optimization homework, now we have small competition. Your task is to optimize 1024-city case as well as possible. Data is here: <a href="https://courses.cs.ut.ee/MTAT.03.238/2022_fall/uploads/Main/1024.txt">1024-city-case</a>.

##### An additional rule is that when you are exporting the image using <a href="https://abercus.github.io/tspvis/">web tool</a>, the label in "Path legal:False", must be True! This confirms that every city has been visited exactly once.

##### Report the best solutions (state-of-the-art) to the Piazza in a public thread, to initialize the competition. In the post describe your approach, report the calculation time, obtained path length and the visualization of the resulting path. The ones who can beat past best solutions will get points (even if someone will outcompete your solution next). To get points to make sure to post new best solutions to corresponding Piazza thread. NB! For the solution to beat previous solutions, it should have either a shorter resulting path or if lengths are comparable the new solution should be considerably faster.

##### Also, very good descriptions of the approach and convergence rates - may get awarded with points upon TA decision

Christofides algorithm:
1. Find MST T of graph
2. Isolate set of odd-degree vertices S
3. Find min weight perfect matching M of S
4. Combine T and M into graph G
5. Generate Eulerian tour of G
6. Generate TSP tour from Eulerian tour

I used Christofides algorithm that guarantees a solution withing a factor of 1.5 of the optimal solution. On average it should give a solution within a factor of 1.1 of the optimal solution.

I further improve the solution of the Christofides algorithm by applying 2-opt improvement.

Best distance: 24186.0089 - time: 10min 14sec

![TSP](https://i.imgur.com/nAMeoeO.png)


In [2]:
%time ex6(1024)

Christofides algorithm distance: 27708.29
Improved to: 24797.35107218375
Improved to: 24779.379317813004
Improved to: 24718.317174707747
Improved to: 24717.381772840836
Improved to: 24683.540935570203
Improved to: 24682.019364361488
Improved to: 24661.77178175191
Improved to: 24653.52272690139
Improved to: 24632.148271465612
Improved to: 24618.922510024113
Improved to: 24613.86029036457
Improved to: 24610.389473547923
Improved to: 24601.52598642358
Improved to: 24593.321398308533
Improved to: 24585.75215392414
Improved to: 24585.142175368466
Improved to: 24582.11912235698
Improved to: 24574.789127704073
Improved to: 24565.520276756662
Improved to: 24530.099988980877
Improved to: 24529.407537394854
Distance after 2-opt improvement: 24529.41
CPU times: total: 4min 53s
Wall time: 7min 53s


Web tool path: [261, 389, 567, 763, 538, 87, 517, 390, 906, 36, 485, 745, 197, 5, 977, 965, 343, 778, 9, 370, 100, 557, 624, 675, 558, 484, 929, 649, 112, 142, 199, 835, 855, 556, 42, 724, 917, 516, 616, 258, 868, 938, 543, 688, 60, 1001, 714, 183, 63, 881, 48, 75, 705, 473, 904, 427, 1006, 487, 927, 509, 372, 1008, 59, 1023, 746, 431, 298, 14, 952, 876, 62, 302, 986, 577, 385, 646, 514, 555, 535, 489, 578, 283, 359, 407, 598, 404, 709, 411, 829, 73, 180, 505, 807, 228, 209, 523, 171, 1021, 381, 811, 249, 222, 542, 802, 115, 46, 813, 963, 163, 568, 305, 823, 354, 680, 685, 153, 405, 551, 597, 659, 104, 937, 378, 281, 424, 861, 884, 384, 920, 98, 471, 376, 645, 901, 203, 968, 573, 232, 114, 337, 461, 272, 909, 239, 150, 130, 394, 790, 167, 981, 276, 525, 103, 235, 441, 704, 65, 653, 698, 784, 367, 759, 983, 949, 928, 534, 622, 102, 89, 362, 17, 738, 1016, 156, 632, 216, 123, 12, 117, 791, 193, 419, 678, 361, 830, 278, 457, 833, 639, 292, 503, 301, 311, 549, 334, 593, 437, 493, 455, 469, 346, 251, 627, 1004, 733, 160, 23, 477, 782, 936, 459, 192, 295, 386, 85, 518, 118, 44, 39, 978, 425, 841, 155, 751, 41, 480, 661, 850, 352, 582, 240, 339, 594, 693, 502, 866, 750, 565, 347, 606, 587, 220, 766, 467, 583, 287, 654, 526, 947, 442, 721, 77, 748, 717, 716, 942, 891, 519, 959, 184, 279, 506, 805, 297, 599, 450, 398, 174, 141, 718, 364, 300, 151, 808, 447, 933, 128, 662, 974, 870, 323, 11, 45, 290, 922, 231, 827, 262, 762, 374, 18, 61, 719, 417, 903, 743, 10, 189, 532, 744, 472, 105, 1019, 1022, 1011, 76, 574, 843, 429, 727, 316, 315, 268, 499, 546, 70, 595, 229, 507, 798, 756, 896, 58, 863, 309, 265, 679, 7, 586, 191, 900, 440, 109, 956, 331, 94, 274, 122, 54, 736, 1010, 998, 233, 412, 121, 463, 476, 676, 647, 809, 607, 640, 241, 217, 202, 621, 629, 847, 652, 428, 681, 177, 934, 78, 666, 663, 227, 207, 911, 851, 162, 492, 351, 734, 908, 707, 420, 691, 562, 754, 80, 794, 964, 69, 93, 168, 877, 559, 839, 561, 16, 195, 226, 182, 188, 864, 86, 40, 826, 554, 52, 205, 566, 993, 838, 320, 973, 992, 682, 125, 610, 584, 131, 332, 71, 438, 391, 496, 277, 569, 648, 545, 19, 326, 344, 737, 697, 500, 779, 776, 987, 795, 923, 783, 872, 379, 710, 804, 939, 399, 831, 919, 366, 617, 512, 824, 529, 925, 777, 822, 414, 55, 967, 165, 402, 846, 215, 1000, 603, 400, 416, 1018, 612, 749, 147, 885, 1009, 687, 1017, 494, 656, 436, 341, 667, 895, 628, 600, 317, 175, 642, 953, 38, 84, 254, 371, 537, 955, 836, 173, 413, 999, 941, 672, 126, 26, 887, 113, 625, 913, 611, 706, 164, 53, 820, 984, 497, 571, 2, 393, 774, 1020, 375, 408, 135, 158, 969, 358, 196, 4, 615, 380, 643, 270, 194, 740, 780, 875, 206, 449, 579, 980, 572, 753, 149, 237, 234, 781, 355, 13, 422, 330, 944, 854, 403, 242, 285, 488, 306, 252, 899, 475, 801, 120, 1007, 741, 458, 576, 619, 914, 972, 350, 760, 225, 510, 668, 761, 27, 64, 858, 985, 329, 348, 848, 482, 521, 101, 960, 319, 68, 796, 145, 303, 860, 591, 388, 520, 134, 880, 169, 29, 90, 324, 267, 418, 596, 35, 137, 633, 318, 637, 97, 690, 528, 764, 815, 498, 547, 1003, 335, 291, 812, 127, 912, 623, 307, 430, 857, 238, 533, 604, 580, 462, 67, 671, 478, 204, 994, 243, 322, 146, 187, 935, 788, 739, 377, 396, 726, 962, 282, 453, 95, 650, 256, 1013, 589, 166, 296, 886, 677, 223, 575, 951, 230, 609, 313, 536, 439, 989, 865, 686, 24, 602, 957, 773, 132, 454, 684, 585, 699, 152, 211, 531, 581, 119, 479, 470, 608, 988, 852, 797, 200, 365, 961, 844, 722, 660, 333, 280, 1, 631, 433, 890, 871, 874, 110, 181, 266, 464, 210, 996, 564, 630, 665, 392, 443, 483, 336, 294, 490, 136, 260, 905, 360, 312, 641, 862, 501, 37, 674, 426, 170, 159, 504, 635, 269, 82, 255, 634, 747, 834, 703, 460, 975, 950, 289, 601, 859, 275, 288, 814, 966, 363, 849, 873, 742, 816, 423, 221, 732, 785, 800, 286, 106, 212, 837, 452, 293, 883, 930, 349, 1012, 626, 825, 271, 915, 728, 34, 190, 673, 902, 435, 560, 74, 771, 752, 474, 144, 83, 56, 867, 828, 382, 432, 22, 395, 700, 328, 185, 696, 28, 468, 111, 971, 31, 940, 201, 775, 948, 495, 99, 540, 96, 129, 224, 793, 713, 176, 368, 882, 563, 508, 79, 670, 72, 720, 451, 765, 898, 997, 926, 644, 530, 550, 708, 767, 466, 401, 47, 345, 618, 548, 853, 373, 406, 273, 257, 208, 731, 527, 353, 757, 253, 30, 588, 907, 723, 715, 369, 116, 179, 138, 821, 711, 421, 976, 21, 491, 888, 735, 51, 148, 892, 486, 43, 92, 1005, 465, 590, 213, 481, 20, 250, 918, 931, 340, 157, 878, 970, 638, 91, 397, 1002, 921, 236, 314, 33, 3, 524, 246, 991, 958, 259, 786, 133, 729, 842, 218, 178, 172, 712, 893, 456, 544, 415, 284, 446, 772, 768, 799, 8, 702, 945, 342, 338, 592, 725, 304, 570, 787, 620, 409, 299, 651, 689, 758, 539, 613, 840, 810, 25, 57, 701, 792, 88, 932, 515, 444, 658, 916, 979, 15, 657, 692, 321, 614, 6, 410, 845, 818, 219, 856, 310, 245, 186, 669, 387, 448, 946, 1014, 327, 513, 244, 552, 605, 356, 803, 995, 894, 789, 511, 683, 143, 819, 695, 357, 308, 32, 541, 1015, 325, 161, 553, 636, 81, 770, 897, 107, 769, 49, 434, 954, 247, 924, 664, 817, 889, 263, 910, 248, 694, 730, 655, 806, 124, 832, 755, 869, 198, 383, 154, 522, 445, 264, 943, 879, 990, 108, 982, 0, 139, 50, 140, 66, 214]

# EX7

#### Provide feedback for the course and homework topics so far

##### which topics were most useful?

Graphs (hw 8), dynamic programming (hw 10), NFA and regular expressions (hw 11)

##### what needs to be covered better in the course?

I think regular expressions deserves to be the main topic of a homework. There are ways to optimise matching, performance of different optimizations could be compared. Advanced regex syntax could also be introduced.

##### are there some topics that would need more practical implementation assignments?

Regular expressions

##### are there some topics that got too much attention (e.g. too boring or otherwise already well known)

None of the topics got too much attention in my opinion.